In [41]:
import pandas as pd
from data_retrievel_and_feature_extraction import uniprot_info as uni
from Bio.PDB import PDBParser

In [42]:
# Gene
gene = "COL11A2"
gene_len = uni.get_sequence_length(gene)

folder = "collagens_proteins"  # collagens_proteins, transmembrane_residues or globular_proteins
# File names
prediction_vs_real_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{folder}\\predictions_vs_real_{folder}.csv"
pathogenic_b_factor_output_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{folder}\\b_factor\\{gene}_pathogenic_b_factor_output_{folder}.csv"
benign_b_factor_output_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{folder}\\b_factor\\{gene}_benign_b_factor_output_{folder}.csv"
pdb_file_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{folder}\\pdb_files\\{gene}.pdb"

# Load the predictions_vs_real data
predictions_vs_real_df = pd.read_csv(prediction_vs_real_path)
predictions_vs_real_df


,gene,variant,prediction,reality
0,COL11A1,G1249V,1,1
1,COL11A1,G1516V,1,1
2,COL11A1,R1733C,1,1
3,COL11A1,G1800C,1,1
4,COL11A1,R1076C,1,1
...,...,...,...,...
1091,COL9A2,G30R,1,0
1092,COL9A2,A336E,1,0
1093,COL9A2,L335V,0,0
1094,COL9A2,R32Q,0,0


In [43]:
# Extract the data for the gene
gene_predictions = predictions_vs_real_df[predictions_vs_real_df["gene"] == gene]

# Add column for position of the variant, according to the "variant" column, which is in the format "L111P"
gene_predictions["position"] = gene_predictions["variant"].apply(lambda x: int(x[1:-1]))

# Add column that contains TP/FP/FN/TN according to the prediction and the reality columns.
gene_predictions["TP/FP/FN/TN"] = gene_predictions.apply(lambda x: "TP" if x["prediction"] == 1 and x["reality"] == 1 else "FP" if x["prediction"] == 1 and x["reality"] == 0 else "FN" if x["prediction"] == 0 and x["reality"] == 1 else "TN", axis=1)

gene_prediction_pathogenic = gene_predictions[gene_predictions["reality"] == 1]
gene_prediction_benign = gene_predictions[gene_predictions["reality"] == 0]

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_20676\708265304.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_predictions["position"] = gene_predictions["variant"].apply(lambda x: int(x[1:-1]))
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_20676\708265304.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_predictions["TP/FP/FN/TN"] = gene_predictions.apply(lambda x: "TP" if x["prediction"] == 1 and x["reality"] == 1 else "FP" if x["prediction"] == 1 and x["reality"] == 0 else "FN" if x[

In [44]:
"""
Create 2 dataframes (benign/pathogenic) that contains values for each position in the gene, according to the TP/FP/FN/TN column. The values will be determined in this way:
For pathogenic variants:
- TP: 1
- FN: 0
- No data, : 3
For benign variants:
- TN: 1
- FP: 0
- No data: 3
# """
benign_b_factor = pd.DataFrame(columns=["position", "b_factor"])
pathogenic_b_factor = pd.DataFrame(columns=["position", "b_factor"])

# For each position in the gene, check if there is data for this position, and insert the value to the right dataframe.
for position in range(1, gene_len+1):
    # Check if there is data for this position in the gene_prediction_pathogenic dataframe
    if position in gene_prediction_pathogenic["position"].values:
        # Check how many instances of this position are there
        position_data = gene_prediction_pathogenic[gene_prediction_pathogenic["position"] == position]
        # If there is only one instance, take the value of the TP/FP/FN/TN column
        if len(position_data) == 1:
            if position_data["TP/FP/FN/TN"].values[0] == "TP":
                # Insert the position and the b_factor value to the pathogenic_b_factor dataframe, without "append" method
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 2]
            elif position_data["TP/FP/FN/TN"].values[0] == "FN":
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 1]
        # If there is more than one instance, check if the predictions are the same, if so- take this value of the TP/FP/FN/TN column. If they are not the same, append "2".
        else:
            # if all the values are the same, take the value of the TP/FP/FN/TN column, and for the other dataframe insert "3" (no data)
            if len(position_data["TP/FP/FN/TN"].unique()) == 1:
                if position_data["TP/FP/FN/TN"].values[0] == "TP":
                    pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 2]
                elif position_data["TP/FP/FN/TN"].values[0] == "FN":
                    pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 1]
            # If the values are not the same, insert "2" for mulitple values
            else:
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 3]       
    # If there is no data for this position, insert "3" for no data
    else:
        pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 4]
        
# For each position in the gene, check if there is data for this position, and insert the value to the right dataframe.
for position in range(1, gene_len+1):
    # Check if there is data for this position in the gene_prediction_pathogenic dataframe
    if position in gene_prediction_benign["position"].values:
        # Check how many instances of this position are there
        position_data = gene_prediction_benign[gene_prediction_benign["position"] == position]
        # If there is only one instance, take the value of the TP/FP/FN/TN column
        if len(position_data) == 1:
            if position_data["TP/FP/FN/TN"].values[0] == "TN":
                # Insert the position and the b_factor value to the pathogenic_b_factor dataframe, without "append" method
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 2]
            elif position_data["TP/FP/FN/TN"].values[0] == "FP":
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 1]
        # If there is more than one instance, check if the predictions are the same, if so- take this value of the TP/FP/FN/TN column. If they are not the same, append "2".
        else:
            # if all the values are the same, take the value of the TP/FP/FN/TN column, and for the other dataframe insert "3" (no data)
            if len(position_data["TP/FP/FN/TN"].unique()) == 1:
                if position_data["TP/FP/FN/TN"].values[0] == "TN":
                    benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 2]
                elif position_data["TP/FP/FN/TN"].values[0] == "FP":
                    benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 1]
            # If the values are not the same, insert "2" for mulitple values
            else:
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 3]       
    # If there is no data for this position, insert "3" for no data
    else:
        benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 4]

In [45]:
# Save the b_factor columns to csv files, without the index column and position column
pathogenic_b_factor

,position,b_factor
0,1,4
1,2,4
2,3,4
3,4,4
4,5,4
...,...,...
1731,1732,4
1732,1733,4
1733,1734,4
1734,1735,4


In [46]:
benign_b_factor

,position,b_factor
0,1,4
1,2,4
2,3,4
3,4,4
4,5,4
...,...,...
1731,1732,4
1732,1733,4
1733,1734,4
1734,1735,4


In [47]:

def get_bfactor_column_for_pdb_file(pdb_file_path, b_factor_per_position):
    """
    Create a series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    :param pdb_file_path: The path to the pdb file
    :param b_factor_per_position: The b_factor values for each position in the gene
    :return: A series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    """
    # Create a DataFrame from the pdb file
    pdb_df = pdb_to_dataframe(pdb_file_path)
    # Create a series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    b_factor_series = pd.Series(index=range(1, len(pdb_df) + 1), name="b_factor")
    for index, row in pdb_df.iterrows():
        # Get the position of the atom in the gene
        position = row["RESIDUESEQ"]
        # Get the b_factor value for this position
        b_factor = b_factor_per_position[b_factor_per_position["position"] == position]["b_factor"].values[0]
        # Insert the b_factor value to the series
        b_factor_series[index+1] = b_factor
    return b_factor_series


def pdb_to_dataframe(pdb_file_path):
    # Create a PDB parser and load the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file_path)

    # Initialize lists to store PDB data
    atom_data = []

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom_record = {
                        'ATOM': atom.get_serial_number(),
                        'NAME': atom.get_name(),
                        'RESNAME': residue.get_resname(),
                        'CHAINID': chain.id,
                        'RESIDUESEQ': residue.id[1],
                        'X': atom.get_coord()[0],
                        'Y': atom.get_coord()[1],
                        'Z': atom.get_coord()[2],
                        'OCCUPANCY': atom.get_occupancy(),
                        'TEMPFACTOR': atom.get_bfactor(),
                        'ELEMENT': atom.element,
                    }
                    atom_data.append(atom_record)

    # Create a DataFrame from the atom data
    df = pd.DataFrame(atom_data)

    return df


In [48]:

pathogenic_b_factor_by_pdb = get_bfactor_column_for_pdb_file(pdb_file_path, pathogenic_b_factor)
benign_b_factor_by_pdb = get_bfactor_column_for_pdb_file(pdb_file_path, benign_b_factor)

# Multiply the b_factor values by 10, to get the same scale as the b_factor values in the pdb file
pathogenic_b_factor_by_pdb = pathogenic_b_factor_by_pdb * 10
benign_b_factor_by_pdb = benign_b_factor_by_pdb * 10
pathogenic_b_factor_by_pdb

1        40.0
2        40.0
3        40.0
4        40.0
5        40.0
         ... 
12104    40.0
12105    40.0
12106    40.0
12107    40.0
12108    40.0
Name: b_factor, Length: 12108, dtype: float64

In [49]:
# Save the b_factor columns to csv files, without the index column
pathogenic_b_factor_by_pdb.to_csv(pathogenic_b_factor_output_path, index=True, header=True)
benign_b_factor_by_pdb.to_csv(benign_b_factor_output_path, index=True, header=True)